In [1]:
import pandas as pd
#from math import radians, cos, sin, asin, sqrt
#from numpy import cos, sin, arcsin, sqrt
from ast import literal_eval
#from haversine import haversine
#from sklearn.metrics.pairwise import haversine_distances
from tqdm import tqdm
import requests
import json
import networkx as nx
import osmnx as ox

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv('CentamapNotNullGeocoded.csv')

In [4]:
df_score = pd.read_csv('ScoreModellingRevised.csv')
#df_score['Building'] = df['Division']

In [5]:
df['geocode'].fillna('[]', inplace=True)

#df.info()

In [6]:
try:
    df['geocode'] = df['geocode'].apply(literal_eval)
except:
    df['geocode'] = None

In [7]:
df.replace('(@) - Figures of individual items...(@) - Figures of individual items are suppressed to ensure data confidentiality.',
          '', inplace = True)

In [8]:
#df_score['geocode'] = df['geocode']

In [9]:
#df_score

In [10]:
df_facilities = pd.read_csv('Facilities.csv')

In [11]:
df_facilities = df_facilities[~df_facilities['latitude'].isnull()]

In [12]:
df_facilities = df_facilities.reset_index()

In [13]:
df_facilities.drop(columns= ['index'], inplace = True)

In [14]:
Leisure
Schools
Home Dept
Hospitals
Acitvity Area
Food 
Electrical/Mechanical
Waste Management 
Open Area for Public 
Post Office
Wifi
Transport
Police
Environmental
Job

SyntaxError: invalid syntax (<ipython-input-14-c2f01464fc99>, line 3)

In [14]:
df_facilities['DataSetValue'].value_counts()

Leisure                  800
Social Welfare           760
Schools                  534
Home Dept                257
Hospitals                177
Acitvity Area            119
Food                     101
Electrical/Mechanical     84
Waste Management          55
Open Area for Public      48
Police                    42
Wifi                      40
Transport                 39
Post Office               36
Environmental             19
Job                       16
Marriage                   4
Name: DataSetValue, dtype: int64

In [22]:
df_school = df_facilities[df_facilities['DataSetValue'] == 'Home Dept']

In [23]:
df_school['Dataset'].value_counts()

Holiday Flats (Licensed)                       45
List of community halls / community centres    42
Hotels (Licensed)                              41
List of licensed club-houses                   37
Holiday Camps (Licensed)                       34
Guesthouses (Licensed)                         33
District Offices and Sub-offices               25
Name: Dataset, dtype: int64

In [24]:
df_school = df_school[df_school['Dataset'] == 'Marine Parks']

In [25]:
df_school['Dataset'].value_counts()

Marine Parks    5
Name: Dataset, dtype: int64

In [ ]:
lst_fac = ['Occasional Child Care Service', 'Long Stay Care Home', 'Residential Special Child Care Centre',
          'Aided Standalone Child Care Centre', 'Mutual Help Child Care Centre',
          'Private and Non-profit making standalone Child Care Centre', 'Special Child Care Centre']

In [ ]:
df_school = df_school[df_school['Dataset'].isin(lst_fac)]

In [26]:
df_school.shape

(5, 118)

In [27]:
df_school = df_school.reset_index()

In [28]:
df_school.drop(columns=['index'], inplace = True)

In [29]:
df_school.head(2)

,Dataset,Facility Name,Address,Name of Recycling Organisations,Types of Material Collection,District,Others,Opening Hours,Telephone,Fax Number / 傳真號碼,...,Country Park,Number of BBQ Pit,Display Barbecue Sites,Country Park/Special Area,Source of water,Tent space / 營位數目,Facilities,geocode,latitude,longitude
0,Marine Parks,Hoi Ha Wan Marine Park,Hoi Ha Wan,NaN,NaN,NaN,NaN,N.A.,NaN,N.A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address_components': [{'long_name': 'Hoi Ha ...,22.470897,114.335629
1,Marine Parks,Sha Chau & Lung Kwu Chau Marine Park,Sha Chau & Lung Kwu Chau,NaN,NaN,NaN,NaN,N.A.,NaN,N.A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address_components': [{'long_name': 'Sha Cha...,22.379017,113.882486


In [30]:
df_sch = pd.DataFrame()
df_sch['building'] = df['Division']
df_sch['geocode'] = df['geocode']
df_sch['lat'] = df['latitude']
df_sch['lon'] = df['longitude']

In [31]:
#df_school.drop_duplicates(subset='Facility Name', inplace = True)

In [32]:
df_school.head(3)

,Dataset,Facility Name,Address,Name of Recycling Organisations,Types of Material Collection,District,Others,Opening Hours,Telephone,Fax Number / 傳真號碼,...,Country Park,Number of BBQ Pit,Display Barbecue Sites,Country Park/Special Area,Source of water,Tent space / 營位數目,Facilities,geocode,latitude,longitude
0,Marine Parks,Hoi Ha Wan Marine Park,Hoi Ha Wan,NaN,NaN,NaN,NaN,N.A.,NaN,N.A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address_components': [{'long_name': 'Hoi Ha ...,22.470897,114.335629
1,Marine Parks,Sha Chau & Lung Kwu Chau Marine Park,Sha Chau & Lung Kwu Chau,NaN,NaN,NaN,NaN,N.A.,NaN,N.A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address_components': [{'long_name': 'Sha Cha...,22.379017,113.882486
2,Marine Parks,Southwest Lantau Marine Park,The Southwestern waters of Lantau,NaN,NaN,NaN,NaN,N.A.,NaN,N.A.,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'address_components': [{'long_name': 'Lantau ...,22.266498,113.941751


In [33]:
a = df_school.shape[0]

In [34]:
for i in range(0,a):
    col = df_school['Facility Name'].iloc[i]
    df_sch[col] = None

In [35]:
for i in df_sch.columns:
    try:
        df_sch[i] = df_sch[i].astype(float)
    except:
        print('Error')

Error
Error


In [36]:
G = ox.graph_from_place('Hong Kong', network_type = "all")
#G1 = ox.graph_from_place('Hong Kong SAR, China', network_type = "all")

In [37]:
'''lat_1 = df_sch['lat'].iloc[10]
lon_1 = df_sch['lon'].iloc[10]
lat_2 = df_school['latitude'].iloc[10]
lon_2 = df_school['longitude'].iloc[10]'''

"lat_1 = df_sch['lat'].iloc[10]\nlon_1 = df_sch['lon'].iloc[10]\nlat_2 = df_school['latitude'].iloc[10]\nlon_2 = df_school['longitude'].iloc[10]"

In [38]:
'''orig_node = ox.nearest_nodes(G, lon_1,  lat_1)
dest_node = ox.nearest_nodes(G, lon_2,  lat_2)'''

'orig_node = ox.nearest_nodes(G, lon_1,  lat_1)\ndest_node = ox.nearest_nodes(G, lon_2,  lat_2)'

In [39]:
#nx.shortest_path_length(G, orig_node, dest_node, weight='length')

In [40]:
#df_sch.iloc[10]

In [41]:
#df_school.iloc[10]

In [42]:
#df_school = df_school.reset_index()

In [43]:
'''%%time  
for i in tqdm(range(0,df_sch.shape[0])):
    lat_1 = df_sch['lat'].iloc[i]
    lon_1 = df_sch['lon'].iloc[i]
    try:
        for j in range(0,df_sch.shape[1]):
            sch_dis = df_sch.columns[j]
            #print(sch_dis)
            for k in range(0,df_school.shape[0]):
                if (df_school['Facility Name'].iloc[k]==sch_dis):
                    lat_2 = df_school['latitude'].iloc[k]
                    lon_2 = df_school['longitude'].iloc[k]
                    r = requests.get(f"http://router.project-osrm.org/route/v1/all/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")
                    json_text = r.json()
                    json_text = json_text['routes'][0]['distance']
                    df_sch[sch_dis].iloc[i] = json_text/1000
    except:
        df_sch[sch_dis].iloc[i] = None'''

'%%time  \nfor i in tqdm(range(0,df_sch.shape[0])):\n    lat_1 = df_sch[\'lat\'].iloc[i]\n    lon_1 = df_sch[\'lon\'].iloc[i]\n    try:\n        for j in range(0,df_sch.shape[1]):\n            sch_dis = df_sch.columns[j]\n            #print(sch_dis)\n            for k in range(0,df_school.shape[0]):\n                if (df_school[\'Facility Name\'].iloc[k]==sch_dis):\n                    lat_2 = df_school[\'latitude\'].iloc[k]\n                    lon_2 = df_school[\'longitude\'].iloc[k]\n                    r = requests.get(f"http://router.project-osrm.org/route/v1/all/{lon_1},{lat_1};{lon_2},{lat_2}?overview=false""")\n                    json_text = r.json()\n                    json_text = json_text[\'routes\'][0][\'distance\']\n                    df_sch[sch_dis].iloc[i] = json_text/1000\n    except:\n        df_sch[sch_dis].iloc[i] = None'

In [44]:
%%time  
for i in tqdm(range(0, df_sch.shape[0])):
    lat_1 = df_sch['lat'].iloc[i]
    lon_1 = df_sch['lon'].iloc[i]
    orig_node = ox.nearest_nodes(G, lon_1,  lat_1)
    for j in range(0,df_sch.shape[1]):
        sch_dis = df_sch.columns[j]
        #print(sch_dis)
        for k in range(0,df_school.shape[0]):
            #print(k)
            if (df_school['Facility Name'].iloc[k]==sch_dis):
                lat_2 = df_school['latitude'].iloc[k]
                lon_2 = df_school['longitude'].iloc[k]
                #print(lat_2)
                try:
                    orig_node = ox.nearest_nodes(G, lon_1,  lat_1)
                    dest_node = ox.nearest_nodes(G, lon_2,  lat_2)
                    json_text = nx.shortest_path_length(G, orig_node, dest_node, weight='distance')
                    df_sch[sch_dis].iloc[i] = json_text
                except:
                    df_sch[sch_dis].iloc[i] = None

100%|████████████████████████████████████████████████████████████████████████████| 5465/5465 [2:00:35<00:00,  1.32s/it]

Wall time: 2h 35s


In [45]:
df_sch.head(5)

,building,geocode,lat,lon,Hoi Ha Wan Marine Park,Sha Chau & Lung Kwu Chau Marine Park,Southwest Lantau Marine Park,The Brothers Marine Park,Tung Ping Chau Marine Park
0,Providence Peak,{'address_components': [{'long_name': 'tower 3...,22.432130,114.202489,44.0,56.0,56.0,56.0,44.0
1,(Sha Tau Kok) residential buildings,{'address_components': [{'long_name': 'Sha Tau...,22.531577,114.203240,44.0,56.0,56.0,56.0,44.0
2,"(Yuen Long Southeast) Tower 1-5, The Reach (Sh...",{'address_components': [{'long_name': 'Shap Pa...,22.437429,114.027660,49.0,36.0,36.0,36.0,49.0
3,"(Yuen Long Southeast) Tower 10-13, The Reach (...",{'address_components': [{'long_name': 'Shap Pa...,22.437429,114.027660,49.0,36.0,36.0,36.0,49.0
4,(Yuen Long Southeast) Grand Del Sol (Fung Cheu...,{'address_components': [{'long_name': 'Fung Ch...,22.440560,114.032987,49.0,36.0,36.0,36.0,49.0


In [46]:
df_sch.to_csv('DistancesMatrixBackup.csv', index = False)

In [47]:
df_sch['Top5'] = None

In [48]:
df_sch.head(2)

,building,geocode,lat,lon,Hoi Ha Wan Marine Park,Sha Chau & Lung Kwu Chau Marine Park,Southwest Lantau Marine Park,The Brothers Marine Park,Tung Ping Chau Marine Park,Top5
0,Providence Peak,{'address_components': [{'long_name': 'tower 3...,22.432130,114.202489,44.0,56.0,56.0,56.0,44.0,None
1,(Sha Tau Kok) residential buildings,{'address_components': [{'long_name': 'Sha Tau...,22.531577,114.203240,44.0,56.0,56.0,56.0,44.0,None


In [49]:
for i in range(4, df_sch.shape[1]):
    try:
        col = df_sch.columns[i]
        #print(col)
        df_sch[col] = round(df_sch[col], 2)
    except:
        print('Error')

Error


In [50]:
for i in df_sch.index:
    df_sch['Top5'].iloc[i] = df_sch.iloc[i].values.tolist()
    df_sch['Top5'].iloc[i] = df_sch['Top5'][i][4:]
    #df_sch['Top5'][i].sort()

In [51]:
df_sch.head(2)

,building,geocode,lat,lon,Hoi Ha Wan Marine Park,Sha Chau & Lung Kwu Chau Marine Park,Southwest Lantau Marine Park,The Brothers Marine Park,Tung Ping Chau Marine Park,Top5
0,Providence Peak,{'address_components': [{'long_name': 'tower 3...,22.432130,114.202489,44.0,56.0,56.0,56.0,44.0,"[44.0, 56.0, 56.0, 56.0, 44.0, None]"
1,(Sha Tau Kok) residential buildings,{'address_components': [{'long_name': 'Sha Tau...,22.531577,114.203240,44.0,56.0,56.0,56.0,44.0,"[44.0, 56.0, 56.0, 56.0, 44.0, None]"


In [52]:
for i in df_sch.index:
    df_sch['Top5'][i] = list(filter(None,df_sch['Top5'][i]))
    df_sch['Top5'][i].sort()
    #df_sch['Top5'].iloc[i] = df_sch['Top5'][i][: 5]

In [53]:
df_sch['Top5_1km'] = None
df_sch['Top5_4km'] = None

In [54]:
for i in df_sch.index:
    df_sch['Top5_1km'].iloc[i] = [number for number in df_sch['Top5'].iloc[i] if number <= 1]
    df_sch['Top5_4km'].iloc[i] = [number for number in df_sch['Top5'].iloc[i] if number <= 4]
    df_sch['Top5_1km'][i].sort()
    df_sch['Top5_4km'][i].sort()

In [55]:
#df_sch['length'] = df_sch['Top5_1m'].apply(len)

In [56]:
for i in df_sch.index:
    df_sch['Top5_1km'].iloc[i] = [1 - x for x in df_sch['Top5_1km'].iloc[i]]
    df_sch['Top5_1km'].iloc[i] =  [round(x,2) for x in df_sch['Top5_1km'].iloc[i]] 
    df_sch['Top5_4km'].iloc[i] = [4 - x for x in df_sch['Top5_4km'].iloc[i]]
    df_sch['Top5_4km'].iloc[i] =  [round(x,2) for x in df_sch['Top5_4km'].iloc[i]] 

In [57]:
df_sch['fisheries_score_1km'] = None
df_sch['fisheries_score_4km'] = None

In [58]:
df_sch.head(2)

,building,geocode,lat,lon,Hoi Ha Wan Marine Park,Sha Chau & Lung Kwu Chau Marine Park,Southwest Lantau Marine Park,The Brothers Marine Park,Tung Ping Chau Marine Park,Top5,Top5_1km,Top5_4km,fisheries_score_1km,fisheries_score_4km
0,Providence Peak,{'address_components': [{'long_name': 'tower 3...,22.432130,114.202489,44.0,56.0,56.0,56.0,44.0,"[44.0, 44.0, 56.0, 56.0, 56.0]",[],[],None,None
1,(Sha Tau Kok) residential buildings,{'address_components': [{'long_name': 'Sha Tau...,22.531577,114.203240,44.0,56.0,56.0,56.0,44.0,"[44.0, 44.0, 56.0, 56.0, 56.0]",[],[],None,None


In [59]:
for i in df_sch.index:
    df_sch['fisheries_score_1km'].iloc[i] = sum(df_sch['Top5_1km'][i])
    df_sch['fisheries_score_4km'].iloc[i] = sum(df_sch['Top5_4km'][i])

In [60]:
df_score['fisheries_score_1km'] = df_sch['fisheries_score_1km']
df_score['fisheries_score_4km'] = df_sch['fisheries_score_4km']
df_score['fisheries_Distance'] = df_sch['Top5']

In [61]:
df_score['fisheries_score_1km'].value_counts()

0    5465
Name: fisheries_score_1km, dtype: int64

In [62]:
df_score['fisheries_score_4km'].value_counts()

0.0    5303
4.0     119
6.0      43
Name: fisheries_score_4km, dtype: int64

In [63]:
#Inequality knowledge

In [64]:
population

NameError: name 'population' is not defined

In [65]:
df_score['lat'] = df_sch['lat']
df_score['lon'] = df_sch['lon']

In [66]:
df_score['fisheries_score_4km'].value_counts()

0.0    5303
4.0     119
6.0      43
Name: fisheries_score_4km, dtype: int64

In [67]:
df_score.to_csv('ScoreModellingRevised.csv', index = False)